In [1]:
%load_ext autoreload
%autoreload 2

In [80]:
import monai.transforms as MT
import torch
import pandas as pd
import numpy as np
import einops
import monai
import os
import glob
import nibabel as nib
import re

import matplotlib.pyplot as plt

from data.fast_dataset import FastTXDS
from bmel.data.fastmri import FastMRIDataset, FastMRITranslateDataset
from sklearn.model_selection import train_test_split
from data.ct_dataset import CTSliceDataset, CTDataset
from data import getds
import myutil
from models import create_model
from tqdm import tqdm

In [73]:
name = 'ct_86_vols'
epoch_ndx = 650
opt = myutil.load_opts(name)
opt.phase = 'test'
opt.epoch = epoch_ndx
opt.num_threads = 0   # test code only supports num_threads = 0
opt.batch_size = 1    # test code only supports batch_size = 1
opt.serial_batches = True  # disable data shuffling; comment this line if results on randomly chosen images are needed.
opt.no_flip = True    # no flip; comment this line if results on flipped images are needed.
opt.display_id = -1   # no visdom display; the test code saves the results to a HTML file.
opt.isTrain = False

In [121]:
_, ds = CTSliceDataset.split()
print(ds.volume_ids)
ds.volume_ids = [80]
ds.preload()
dl = torch.utils.data.DataLoader(ds, batch_size=opt.batch_size, num_workers=0)

[80, 1, 75, 25, 15, 60, 13, 21, 5, 71, 65, 68, 57, 78, 66, 70, 36, 83]
loading volume_id=80[1]
loading volume_id=80[4]


In [122]:
model = create_model(opt)
model.setup(opt)
model.eval()

initialize network with normal
model [Pix2PixModel] was created
loading the model from ./checkpoints/ct_86_vols/650_net_G.pth
---------- Networks initialized -------------
[Network G] Total number of parameters : 29.238 M
-----------------------------------------------


In [136]:
preds = {volume_id: {} for volume_id in ds.volume_ids}

for sample_ndx, sample in enumerate(tqdm(ds)):
    volume_id = sample['volume_id']
    slc_ndx = sample['slc_ndx']
    sample['A'] = sample['A'].unsqueeze(0)
    sample['B'] = sample['B'].unsqueeze(0)
    if volume_id != 80: continue
    model.set_input(sample)
    model.test()
    visuals = model.get_current_visuals()
    for k in visuals:
        visuals[k] = visuals[k][0,0].cpu()
    preds[volume_id][slc_ndx] = visuals['fake_B']
    # fig, ax = plt.subplots(1, 4, figsize=(40, 10))
    # im = ax[0].imshow(visuals['real_A'], 'gray'); plt.colorbar(im, ax=ax[0])
    # im = ax[1].imshow(visuals['fake_B'], 'gray'); plt.colorbar(im, ax=ax[1])
    # im = ax[2].imshow(visuals['real_B'], 'gray'); plt.colorbar(im, ax=ax[2])
    # im = ax[3].imshow(visuals['real_B'] - visuals['fake_B'], 'gray'); plt.colorbar(im, ax=ax[3])
    # plt.show()
    

  5%|▍         | 135/2726 [00:00<00:07, 351.55it/s]


In [141]:
pred = preds[80]
assert all(slc_ndx in pred for slc_ndx in range(len(pred)))
pred = torch.stack([pred[slc_ndx] for slc_ndx in sorted(pred)])
# ds.volumes[80][4], ds.volumes[80][1]

In [147]:
pred.max(), ds.volumes[80][4][0].max(), ds.volumes[80][1][0].max()

(tensor(0.8984), tensor(1.), tensor(1.))

In [150]:
torch.nn.functional.mse_loss(ds.volumes[80][1][0], ds.volumes[80][4][0])

tensor(3.4204e-05)